In [1]:
import collections
import mxnet as mx
import numpy as np
from mxnet import autograd, gluon, init, metric, nd
from mxnet.contrib import text
from mxnet.gluon import loss as gloss, nn, rnn
from mxnet.gluon import data as gdata, loss as gloss, nn
from sklearn import metrics
import os
import random
import zipfile
import re
import pickle
import csv
import time
import matplotlib.pyplot as plt

In [4]:
def top_n_accuracy(preds, truths, n, model):
    best_n = np.argsort(preds, axis=1)[:,-n:]
    ts = truths
    successes = 0
    for i in range(len(ts)):
        if ts[i] in [model.classes_[line] for line in best_n[i,:]]:
            successes += 1
    return float(successes)/len(ts)

In [5]:
def count_token(train_tokenized, token_counter):
    for sample in train_tokenized:
        for token in sample:
            if token not in token_counter:
                token_counter[token] = 1
            else:
                token_counter[token] += 1
    return token_counter

def encode_samples(tokenized_samples, vocab):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in vocab.token_to_idx:
                feature.append(vocab.token_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)         
    return features

def pad_samples(features, maxlen=1000, PAD=0):
    padded_features = []
    for feature in features:
        if len(feature) > maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            # 添加 PAD 符号使每个序列等长（长度为 maxlen ）。
            while len(padded_feature) < maxlen:
                padded_feature.append(PAD)
        padded_feature  = [0,0,0,0] + padded_feature + [0,0,0,0]
        padded_features.append(padded_feature)
    return padded_features

In [6]:
TRUE_LABEL = [
    'Transient', 
    'False Alarm', 
    'Won\'t Fix', 
    'Unable To Reproduce', 
    'Customer Error',
    'Won\'t fix',
    'By Design',
]
TRUE_LABEL = list(set([i.upper() for i in TRUE_LABEL]))

LABELDICT = {}
for i in range(len(TRUE_LABEL)):
    LABELDICT[TRUE_LABEL[i]] = i
LABELDICT['NeedFixed'.upper()] = len(TRUE_LABEL)
print(LABELDICT)    

def getBinaryLabel(x):
    if x.upper() in TRUE_LABEL:
        return 1
    
    return 0

def getMultiLabel(x):
    return LABELDICT[x.upper()]

{'UNABLE TO REPRODUCE': 0, 'BY DESIGN': 1, "WON'T FIX": 2, 'FALSE ALARM': 3, 'TRANSIENT': 4, 'CUSTOMER ERROR': 5, 'NEEDFIXED': 6}


In [7]:
feature_columns = [
    'MonitorId',
    'OccurringEnvironment',
    'TenantSilos',
    'OccurringDatacenter',
    'OccurringDeviceName',
    'OccurringDeviceGroup',
    'OccurringServiceInstanceId',
    'RaisingEnvironment',
    'RaisingDatacenter',
    'RaisingDeviceName',
    'RaisingDeviceGroup',
    'IncidentType',
    'SourceType',
    'SourceOrigin',
    'variable'
] 

In [8]:
with open('word_dict.pkl','rb') as file:
    twitter_dict=pickle.load(file)

In [9]:
def clean_str(w):
    if ' ' not in w: 
        if w not in twitter_dict:
            return w
        else:
            return None
    else:
        return (' ').join([x for x in w.split(' ') if x not in twitter_dict])

In [11]:
def project_do(project_name, dropout_rate): 
    print('====')
    print(project_name)
    print('====')
    
    csv_write = csv.writer(open('DeepTIP-Attention.csv','a',newline=''), dialect='excel')
    def count_token(train_tokenized, token_counter):
        for sample in train_tokenized:
            for token in sample:
                if token not in token_counter:
                    token_counter[token] = 1
                else:
                    token_counter[token] += 1
        return token_counter
    
    def encode_samples(tokenized_samples, vocab):
        features = []
        for sample in tokenized_samples:
            feature = []
            for token in sample:
                if token in vocab.token_to_idx:
                    feature.append(vocab.token_to_idx[token])
                else:
                    feature.append(0)
            features.append(feature)         
        return features

    def pad_samples(features, maxlen=1000, PAD=0):
        padded_features = []
        for feature in features:
            if len(feature) > maxlen:
                padded_feature = feature[:maxlen]
            else:
                padded_feature = feature
                # 添加 PAD 符号使每个序列等长（长度为 maxlen ）。
                while len(padded_feature) < maxlen:
                    padded_feature.append(PAD)
            padded_feature  = [0,0,0,0] + padded_feature + [0,0,0,0]
            padded_features.append(padded_feature)
        return padded_features
    
    class TextCNN(nn.Block):
        def __init__(self, vocab, embedding_size, ngram_kernel_sizes, nums_channels, num_outputs, max_len_dict, sequence_len, **kwargs):
            
            super(TextCNN, self).__init__(**kwargs)
            self.ngram_kernel_sizes = ngram_kernel_sizes
            self.nums_channels = nums_channels
            self.embedding = nn.Embedding(len(vocab), embedding_size)            
            self.embedding_MonitorId = nn.Embedding(max_len_dict['MonitorId'], 16)
            self.embedding_OccurringEnvironment = nn.Embedding(max_len_dict['OccurringEnvironment'], 16)
            self.embedding_TenantSilos = nn.Embedding(max_len_dict['TenantSilos'], 16)
            self.embedding_OccurringDatacenter = nn.Embedding(max_len_dict['OccurringDatacenter'], 4)
            self.embedding_OccurringDeviceName = nn.Embedding(max_len_dict['OccurringDeviceName'], 16)
            self.embedding_OccurringDeviceGroup = nn.Embedding(max_len_dict['OccurringDeviceGroup'], 4)
            self.embedding_OccurringServiceInstanceId = nn.Embedding(max_len_dict['OccurringServiceInstanceId'], 4)
            self.embedding_RaisingEnvironment = nn.Embedding(max_len_dict['RaisingEnvironment'], 4)
            self.embedding_RaisingDatacenter = nn.Embedding(max_len_dict['RaisingDatacenter'], 4)
            self.embedding_RaisingDeviceName = nn.Embedding(max_len_dict['RaisingDeviceName'], 4)
            self.embedding_RaisingDeviceGroup = nn.Embedding(max_len_dict['RaisingDeviceGroup'], 4)
            self.embedding_IncidentType = nn.Embedding(max_len_dict['IncidentType'], 4)
            self.embedding_SourceType = nn.Embedding(max_len_dict['SourceType'], 4)
            self.embedding_SourceOrigin = nn.Embedding(max_len_dict['SourceOrigin'], 4)
            self.embedding_variable = nn.Embedding(max_len_dict['variable'], 64)
            
            for i in range(len(ngram_kernel_sizes)):
                conv = nn.Conv1D(nums_channels[i],
                    kernel_size=ngram_kernel_sizes[i],
                    strides=1,
                    activation='relu')
                pool = nn.GlobalMaxPool1D()
                setattr(self, f'conv_{i}', conv) 
                setattr(self, f'pool_{i}', pool)

            self.temp_pool = nn.GlobalMaxPool1D()
            self.dropout = nn.Dropout(0.5)
            self.attention = self.attention_model(sequence_len)
            self.dropout2 = nn.Dropout(dropout_rate)
            self.dense = nn.Dense(32)  
            self.decoder = nn.Dense(num_outputs)
       
        def forward(self, inputs, context, MonitorId, OccurringEnvironment, OccurringDeviceName, IncidentType, SourceType, SourceOrigin, variable):

            embeddings = self.embedding(inputs)
            embedding_context = self.embedding(context)
            embedd_MonitorId = self.embedding_MonitorId(MonitorId)
            embedd_OccurringEnvironment = self.embedding_OccurringEnvironment(OccurringEnvironment)
            embedd_TenantSilos = self.embedding_TenantSilos(TenantSilos)
            embedd_OccurringDatacenter = self.embedding_OccurringDatacenter(OccurringDatacenter)
            embedd_OccurringDeviceName = self.embedding_OccurringDeviceName(OccurringDeviceName)
            embedd_OccurringDeviceGroup = self.embedding_OccurringDeviceGroup(OccurringDeviceGroup)
            embedd_OccurringServiceInstanceId = self.embedding_OccurringServiceInstanceId(OccurringServiceInstanceId)
            embedd_RaisingEnvironment = self.embedding_RaisingEnvironment(RaisingEnvironment)
            embedd_RaisingDatacenter = self.embedding_RaisingDatacenter(RaisingDatacenter)
            embedd_RaisingDeviceName = self.embedding_RaisingDeviceName(RaisingDeviceName)
            embedd_RaisingDeviceGroup = self.embedding_RaisingDeviceGroup(RaisingDeviceGroup)
            embedd_IncidentType = self.embedding_IncidentType(IncidentType)
            embedd_SourceType = self.embedding_SourceType(SourceType)
            embedd_SourceOrigin = self.embedding_SourceOrigin(SourceOrigin)
            embedd_variable = self.embedding_variable(variable)
            
            # TARGET
            embeddings = embeddings.transpose((0,2,1)) 
            encoding = [
                nd.flatten(self.get_pool(i)(self.get_conv(i)(embeddings)))
                for i in range(len(self.ngram_kernel_sizes))]
            encoding = nd.concat(*encoding, dim=1)
            
            # CONTEXT
            context_conved = []
            for line in embedding_context.transpose((1,0,3,2)):
                c_encoding = [
                            nd.flatten(self.get_pool(i)(self.get_conv(i)(line)))
                            for i in range(len(self.ngram_kernel_sizes))]
                c_encoding = nd.concat(*c_encoding, dim=1)
                context_conved.append(nd.expand_dims(c_encoding,axis=1))
            
            context_conved.append(nd.expand_dims(encoding,axis=1))
            context_conved = nd.concat(*context_conved, dim=1).transpose((1,0,2))            
            query_encoding = nd.broadcast_axis(encoding.expand_dims(0), axis=0, size=context_conved.shape[0])
            
            enc_and_dec_states = nd.concat(context_conved, query_encoding, dim=2)
            e = self.attention(enc_and_dec_states)
            alpha = nd.softmax(e, axis=0) 
            context_vector = (alpha * context_conved).sum(axis=0) 
            
            outputs = self.dense(self.dropout(
                nd.concat(
                encoding,
                context_vector,
                embedd_MonitorId,
                embedd_OccurringEnvironment,
                embedd_TenantSilos,
                embedd_OccurringDatacenter,
                embedd_OccurringDeviceName,
                embedd_OccurringDeviceGroup,
                embedd_OccurringServiceInstanceId,
                embedd_RaisingEnvironment,
                embedd_RaisingDatacenter,
                embedd_RaisingDeviceName,
                embedd_RaisingDeviceGroup,
                embedd_IncidentType,
                embedd_SourceType,
                embedd_SourceOrigin,
                embedd_variable,
                dim=1)               
            ))
            
            outs = self.decoder(self.dropout2(outputs))
            return outs
        
        def get_conv(self, i):
            return getattr(self, f'conv_{i}')
        
        def get_pool(self, i):
            return getattr(self, f'pool_{i}')
        
        def get_conv_c(self, i):
            return getattr(self, f'conv_c_{i}')
        
        def get_pool_c(self, i):
            return getattr(self, f'pool_c_{i}')
        
        def attention_model(self, attention_size):
            model = nn.Sequential()
            model.add(nn.Dense(attention_size, activation='tanh', use_bias=False, flatten=False),
                      nn.Dense(1, use_bias=False, flatten=False))
            return model
    
    def eval_model(train_iter):
        l_sum = 0
        l_n = 0
        pred_probs = []
        labels_result = []
        for data, context, label, weights, MonitorId, OccurringEnvironment, OccurringDeviceName, IncidentType, SourceType, SourceOrigin, variable in train_iter:
            X = data.as_in_context(ctx)
            C = context.as_in_context(ctx)
            y = label.as_in_context(ctx).T
            
            MonitorId = MonitorId.as_in_context(ctx)
            OccurringEnvironment = OccurringEnvironment.as_in_context(ctx)
            TenantSilos = TenantSilos.as_in_context(ctx)
            OccurringDatacenter = OccurringDatacenter.as_in_context(ctx)
            OccurringDeviceName = OccurringDeviceName.as_in_context(ctx)
            OccurringDeviceGroup = OccurringDeviceGroup.as_in_context(ctx)
            OccurringServiceInstanceId = OccurringServiceInstanceId.as_in_context(ctx)
            RaisingEnvironment = RaisingEnvironment.as_in_context(ctx)
            RaisingDatacenter = RaisingDatacenter.as_in_context(ctx)
            RaisingDeviceName = RaisingDeviceName.as_in_context(ctx)
            RaisingDeviceGroup = RaisingDeviceGroup.as_in_context(ctx)
            IncidentType = IncidentType.as_in_context(ctx)
            SourceType = SourceType.as_in_context(ctx)
            SourceOrigin = SourceOrigin.as_in_context(ctx)
            variable = variable.as_in_context(ctx)                      
            
            y_pred = net(X, C, MonitorId, OccurringEnvironment, OccurringDeviceName, IncidentType, SourceType, SourceOrigin, variable)
            l = loss(y_pred, y)
            labels_result.extend(list(y.astype('int').asnumpy()))
            
            for line in y_pred:         
                pred_probs.extend(list(nd.softmax(line)[1].astype('float').asnumpy()))
                
        fpr, tpr, thresholds = metrics.roc_curve(labels_result, pred_probs)
        roc_auc = metrics.auc(fpr, tpr)
        
        return 1, roc_auc
    
    def eval_model_test(train_iter):
        l_sum = 0
        l_n = 0
        pred_result = []
        labels_result = []
        pred_probs = []
        
        for data, context, label, weights, MonitorId, OccurringEnvironment, OccurringDeviceName, IncidentType, SourceType, SourceOrigin, variable in train_iter:
            X = data.as_in_context(ctx)
            C = context.as_in_context(ctx)
            y = label.as_in_context(ctx).T
            
            MonitorId = MonitorId.as_in_context(ctx)
            OccurringEnvironment = OccurringEnvironment.as_in_context(ctx)
            TenantSilos = TenantSilos.as_in_context(ctx)
            OccurringDatacenter = OccurringDatacenter.as_in_context(ctx)
            OccurringDeviceName = OccurringDeviceName.as_in_context(ctx)
            OccurringDeviceGroup = OccurringDeviceGroup.as_in_context(ctx)
            OccurringServiceInstanceId = OccurringServiceInstanceId.as_in_context(ctx)
            RaisingEnvironment = RaisingEnvironment.as_in_context(ctx)
            RaisingDatacenter = RaisingDatacenter.as_in_context(ctx)
            RaisingDeviceName = RaisingDeviceName.as_in_context(ctx)
            RaisingDeviceGroup = RaisingDeviceGroup.as_in_context(ctx)
            IncidentType = IncidentType.as_in_context(ctx)
            SourceType = SourceType.as_in_context(ctx)
            SourceOrigin = SourceOrigin.as_in_context(ctx)
            variable = variable.as_in_context(ctx)                      
            
            y_pred= net(X, C, MonitorId, OccurringEnvironment, OccurringDeviceName, IncidentType, SourceType, SourceOrigin, variable)           
            l = loss(y_pred, y)
            pred_result.extend(list(nd.argmax(nd.softmax(y_pred), axis=1).astype('int').asnumpy()))
            labels_result.extend(list(y.astype('int').asnumpy()))
            for line in y_pred:         
                pred_probs.extend(list(nd.softmax(line)[1].astype('float').asnumpy()))
        
        return metrics.precision_score(labels_result, pred_result, average='macro'), \
                    metrics.recall_score(labels_result, pred_result, average='macro'),\
                    metrics.f1_score(labels_result, pred_result, average='macro'),\
                    metrics.accuracy_score(labels_result, pred_result), \
                    pred_result,pred_probs,labels_result
    
    ROOTPATH = "PATH"
    samples = pickle.load(open(ROOTPATH + project_name + '_train_title_summary.pkl','rb'))
    samples_test = pickle.load(open(ROOTPATH + project_name + '_test_title_summary.pkl','rb'))
    
    train = {}
    test = {}
    
    samples['Label_Multi'] = samples['Label'].apply(lambda x: getMultiLabel(x))
    samples_test['Label_Multi'] = samples_test['Label'].apply(lambda x: getMultiLabel(x))
    
    samples['Label_Binary'] = samples['Label'].apply(lambda x: getBinaryLabel(x))
    samples_test['Label_Binary'] = samples_test['Label'].apply(lambda x: getBinaryLabel(x))
    
    samples['variable'] = samples['variable'].apply(lambda x: clean_str(str(x)))
    samples_test['variable'] = samples_test['variable'].apply(lambda x: clean_str(str(x)))
    
    train['sentences'] = list(samples['tokenized'])
    train['labels'] = list(samples['Label_Binary'])
    
    # multi weight for train
    weight_dict = {}
    total = len(samples['Label_Multi'])
    min_sample = samples['Label_Multi'].value_counts().min()
    for index,x in samples['Label_Multi'].value_counts().iteritems():
        weight_dict[index] = 1 - float(x) / total
    
    weightMatrix = nd.array([weight_dict[x] for x in weight_dict])
    weightMatrix = nd.softmax(weightMatrix)
    
    i=0
    for x in weight_dict:
        weight_dict[x] = weightMatrix[i].astype('float').asnumpy()[0]
        i += 1
    
    train['weights'] = []
    for line in samples['Label_Multi']:
        train['weights'].append(weight_dict[line])
    train['weights'] =  nd.array(train['weights'])
                        
    test['sentences'] = list(samples_test['tokenized']) 
    test['labels'] = list(samples_test['Label_Binary'])
    
    test['weights'] = []
    for line in samples_test['Label_Multi']:  
         test['weights'].append(weight_dict[line])
    test['weights'] = nd.array(test['weights'])
    
    max_len = 80
    while (np.sum(np.array(list(map(len,train['sentences'])))<max_len)/len(train['sentences'])<0.95):
        max_len += 10   
    
    for c in feature_columns:
        tmp_dict = {}
        idx = 0
        for x in list(samples[c].drop_duplicates()):
            tmp_dict[x] = idx
            idx += 1
            
        for x in list(samples_test[c].drop_duplicates()):
            if x not in tmp_dict:
                tmp_dict[x] = idx
                idx += 1
            
        train[c] = nd.array(list(samples[c].replace(tmp_dict)))
        test[c] = nd.array(list(samples_test[c].replace(tmp_dict)))

    token_counter = collections.Counter()
    token_counter = count_token(train['sentences'], token_counter)
    vocab = text.vocab.Vocabulary(token_counter, unknown_token='<unk>', min_freq=2, reserved_tokens=None, most_freq_count=30000)
    
    ctx = mx.gpu(1)
    from gensim.models import FastText
    model = FastText(train['sentences'] + test['sentences'], min_count=1, size=200, iter=10, workers=8)
    w2v = []
    for line in vocab.token_to_idx:
        if line in model.wv:
            w2v.append(model.wv[line])
        else:
            w2v.append(np.zeros(200))
    w2v = nd.array(w2v)
    print('w2v success', w2v.shape)
    
    train['features'] = encode_samples(train['sentences'], vocab)
    train['features'] = nd.array(pad_samples(train['features'], max_len, 0))
                                
    test['features'] = encode_samples(test['sentences'], vocab)
    test['features'] = nd.array(pad_samples(test['features'], max_len, 0))
    
    max_len_dict = {}
    for c in feature_columns:
        max_len = int(np.max(train[c]).asnumpy()) 
        if max_len <int(np.max(test[c]).asnumpy()) :
            max_len = int(np.max(test[c]).asnumpy())
        max_len_dict[c] = max_len
    print('encode_samples success')
    
    sequence_length = 10
    
    train['samples'] = []
    for index,line in enumerate(train['features']):
        if index<=sequence_length:
            continue
        
        if index % 100000==0:
            print(index)
        
        cur_samples = []
        for i in range(-sequence_length,-1,1):
            
            if index + i >= 0 and index + i<len(train['features']):
                cur_samples.append(nd.expand_dims(train['features'][index + i],axis=0))
        
        train['samples'].append(nd.concat(*cur_samples,dim=0))
    
    test['samples'] = []
    for index,line in enumerate(test['features']):
        if index<=sequence_length:
            continue
        
        if index % 100000==0:
            print(index)
        
        cur_samples = []
        for i in range(-sequence_length,-1,1):
            if index + i >= 0 and index + i<len(test['features']):
                cur_samples.append(nd.expand_dims(test['features'][index + i],axis=0))
        test['samples'].append(nd.concat(*cur_samples,dim=0))
    
    lr = 0.001
    num_epochs = 10
    sentence_hidden = 32
    batch_size = 512
    embed_size = 200
    ngram_kernel_sizes = [3,4,5]
    nums_channels = [100,100,100]
    
    net = TextCNN(vocab, embed_size, ngram_kernel_sizes, nums_channels, 2, max_len_dict, sequence_length)
    net.initialize(init.Xavier(), ctx=ctx)
    net.embedding.weight.set_data(w2v.as_in_context(ctx))
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr})
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    train_iter = gdata.DataLoader(gdata.ArrayDataset(
        train['features'][sequence_length + 1:],
        train['samples'], 
        train['labels'][sequence_length + 1:],
        train['weights'][sequence_length + 1:], 
        train['MonitorId'][sequence_length + 1:],
        train['OccurringEnvironment'][sequence_length + 1:],
        train['TenantSilos'][sequence_length + 1:],
        train['OccurringDatacenter'][sequence_length + 1:],
        train['OccurringDeviceName'][sequence_length + 1:],
        train['OccurringDeviceGroup'][sequence_length + 1:],
        train['OccurringServiceInstanceId'][sequence_length + 1:],
        train['RaisingEnvironment'][sequence_length + 1:],
        train['RaisingDatacenter'][sequence_length + 1:],
        train['RaisingDeviceName'][sequence_length + 1:],
        train['RaisingDeviceGroup'][sequence_length + 1:],
        train['IncidentType'][sequence_length + 1:],
        train['SourceType'][sequence_length + 1:],
        train['SourceOrigin'][sequence_length + 1:],
        train['variable'][sequence_length + 1:]
    ), batch_size, shuffle=True)
    
    test_iter = gdata.DataLoader(gdata.ArrayDataset(
        test['features'][sequence_length + 1:],
        test['samples'], 
        test['labels'][sequence_length + 1:],
        test['weights'][sequence_length + 1:], 
        test['MonitorId'][sequence_length + 1:],
        test['OccurringEnvironment'][sequence_length + 1:],
        test['TenantSilos'][sequence_length + 1:],
        test['OccurringDatacenter'][sequence_length + 1:],
        test['OccurringDeviceName'][sequence_length + 1:],
        test['OccurringDeviceGroup'][sequence_length + 1:],
        test['OccurringServiceInstanceId'][sequence_length + 1:],
        test['RaisingEnvironment'][sequence_length + 1:],
        test['RaisingDatacenter'][sequence_length + 1:],
        test['RaisingDeviceName'][sequence_length + 1:],
        test['RaisingDeviceGroup'][sequence_length + 1:],
        test['IncidentType'][sequence_length + 1:],
        test['SourceType'][sequence_length + 1:],
        test['SourceOrigin'][sequence_length + 1:],
        test['variable'][sequence_length + 1:]
    ), batch_size, shuffle=True)
    
    best_auc = 0
    start = time.clock()
    for epoch in range(1, num_epochs + 1):
        l_sum = 0
        l_n = 0
        for data, context, label, weight, MonitorId, OccurringEnvironment, OccurringDeviceName, IncidentType, SourceType, SourceOrigin, variable in train_iter:
            X = data.as_in_context(ctx)
            C = context.as_in_context(ctx)
            y = label.as_in_context(ctx).T
            w = weight.as_in_context(ctx).T
            
            MonitorId = MonitorId.as_in_context(ctx)
            OccurringEnvironment = OccurringEnvironment.as_in_context(ctx)
            TenantSilos = TenantSilos.as_in_context(ctx)
            OccurringDatacenter = OccurringDatacenter.as_in_context(ctx)
            OccurringDeviceName = OccurringDeviceName.as_in_context(ctx)
            OccurringDeviceGroup = OccurringDeviceGroup.as_in_context(ctx)
            OccurringServiceInstanceId = OccurringServiceInstanceId.as_in_context(ctx)
            RaisingEnvironment = RaisingEnvironment.as_in_context(ctx)
            RaisingDatacenter = RaisingDatacenter.as_in_context(ctx)
            RaisingDeviceName = RaisingDeviceName.as_in_context(ctx)
            RaisingDeviceGroup = RaisingDeviceGroup.as_in_context(ctx)
            IncidentType = IncidentType.as_in_context(ctx)
            SourceType = SourceType.as_in_context(ctx)
            SourceOrigin = SourceOrigin.as_in_context(ctx)
            variable = variable.as_in_context(ctx)
            
            with autograd.record():
                y_hat = net(X, C, MonitorId, OccurringEnvironment, OccurringDeviceName, IncidentType, SourceType, SourceOrigin, variable)
                l = loss(y_hat, y, w)
            
            l.backward()
            trainer.step(batch_size, ignore_stale_grad=True)
        
        train_loss, train_auc = eval_model(train_iter)
        val_loss, val_auc = eval_model(test_iter)
        if val_auc>=best_auc:
            net.save_parameters('DeepTIP.model')
            best_auc = val_auc
        print('epoch %d, train loss %.4f, auc %.2f; val loss %.6f, auc %.4f' 
              % (epoch, train_loss, train_auc, val_loss, val_auc))
    
    end = time.clock()
    print('train success')
   
    net.load_parameters('DeepTIP.model')
    start_p = time.clock()
    pre, recall, f1, acc1,pred_result,pred_probs,labels_result = eval_model_test(test_iter)
    end_p = time.clock()
    
    fpr, tpr, thresholds = metrics.roc_curve(labels_result, pred_probs)
    roc_auc = metrics.auc(fpr, tpr)
    lw = 2
    plt.figure(figsize=(8, 5))
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)  ###假正率为横坐标，真正率为纵坐标做曲线
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('DeepTip-Binary')
    plt.legend(loc="lower right")
    plt.show()
    
    csv_write.writerow([project_name,'DeepTip', roc_auc, pre, recall, f1, acc1,end-start,end_p-start_p])
    pickle.dump([pred_result, labels_result],open('result/' + project_name + '_dl_em.proba.pkl','wb'))